In [1]:
import ray
import os
import time
import numpy as np

%env RAY_DASHBOARD_DEBUG = True

ray.init()

2020-01-02 16:00:42,070	INFO resource_spec.py:216 -- Starting Ray with 3.03 GiB memory available for workers and up to 1.53 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


env: RAY_DASHBOARD_DEBUG=True


2020-01-02 16:00:42,444	INFO services.py:1096 -- View the Ray dashboard at localhost:8267.


{'node_ip_address': '10.1.10.91',
 'redis_address': '10.1.10.91:30378',
 'object_store_address': '/tmp/ray/session_2020-01-02_16-00-42_062493_7423/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-01-02_16-00-42_062493_7423/sockets/raylet',
 'webui_url': 'localhost:8267',
 'session_dir': '/tmp/ray/session_2020-01-02_16-00-42_062493_7423'}

### Debug blocked actor creation tasks

In [2]:
@ray.remote(resources={"Custom": 1})
class A(object):
    def __init__(self):
        pass
    
    def f(self):
        return 0
        
@ray.remote
class B(object):
    def __init__(self):
        self.a = A.remote()
        
    def f(self):
        return ray.get(self.a.f.remote())

b = B.remote()

try:
    ray.get(b.f.remote(), timeout=2)
except ray.exceptions.RayTimeoutError:
    print("Session hangs because actor A cannot be created. ")

2020-01-02 16:01:02,482	WARNING worker.py:1056 -- The actor or task with ID ffffffffffffffff2512146c0100 is infeasible and cannot currently be scheduled. It requires {Custom: 1.000000}, {CPU: 1.000000} for execution and {Custom: 1.000000}, {CPU: 1.000000} for placement, however there are no nodes in the cluster that can provide the requested resources. To resolve this issue, consider reducing the resource requests of this task or add nodes that can fit the task.


Session hangs because actor A cannot be created. 


### Inspect node memory usage

In [3]:
@ray.remote
class C(object):
    def __init__(self):
        pass
    
    def f(self):
        return ray.put(np.random.random((100, 100)))
    
@ray.remote
class D(object):
    def __init__(self):
        self.c = C.remote()
        
    def f(self):
        for _ in range(20):
            time.sleep(0.5)
            object_id = ray.get(self.c.f.remote())
            assert isinstance(ray.get(object_id), np.ndarray)            
        
d = D.remote()
_ = ray.get(d.f.remote())

### Inspect worker memory usage

In [4]:
@ray.remote
class E(object):
    def __init__(self):
        pass
    
    def f(self):
        return "hello world"
    
@ray.remote
class F(object):
    def __init__(self):
        self.e = E.remote()
        self.object_ids = []
        
    def f(self):
        for _ in range(20):
            time.sleep(0.5)
            self.object_ids.append(self.e.f.remote())
        
f = F.remote()
_ = ray.get(f.f.remote())